In [1]:
import jinja2
from  anthropic import AnthropicBedrock
from compiler.core import Compiler
from tracing_client import TracingClient
from policies import common, handlers

In [2]:
jinja_env = jinja2.Environment()
compiler = Compiler("botbuild/app_schema", "botbuild/app_schema")
client = TracingClient(AnthropicBedrock(aws_profile="dev", aws_region="us-west-2"))

In [3]:
application_description = "Bot that always responds with a greeting."
application_description = "Bot that writes poems about cars."

In [4]:
import os

DATASET_DIR = "evals/dataset.min/"
files = os.listdir(DATASET_DIR)
drizzle_file = "car_poem_drizzle_schema.ts"
typescript_file = "car_poem_typescript_schema.ts"
with open(os.path.join(DATASET_DIR, drizzle_file), "r", encoding="utf-8") as f:
    drizzle_schema = f.read()
with open(os.path.join(DATASET_DIR, typescript_file), "r", encoding="utf-8") as f:
    typespec_definitions = f.read()

content = jinja_env.from_string(handlers.PROMPT).render(application_description=application_description, function_name="getCarPoem", drizzle_schema=drizzle_file, typespec_definitions=typespec_definitions)
message = {"role": "user", "content": content}

In [5]:
with handlers.HandlerTaskNode.platform(client, compiler, jinja_env):
    output = handlers.HandlerTaskNode.run([message])

Langfuse client is disabled since no public_key was provided as a parameter or environment variable 'LANGFUSE_PUBLIC_KEY'. See our docs: https://langfuse.com/docs/sdk/python/low-level-sdk#initialize-client


In [6]:
output.output

AttributeError("'str' object has no attribute 'items'")

In [7]:
root = handlers.HandlerTaskNode(output)

In [ ]:
with handlers.HandlerTaskNode.platform(client, compiler, jinja_env):
    output = root.run()


In [8]:
with handlers.HandlerTaskNode.platform(client, compiler, jinja_env):
    solution = common.bfs(root)

In [9]:
solution.score, solution.depth

(0, 0)

In [10]:
for node in solution.parent.children:
    print(node.is_successful, node.data.output)

AttributeError: 'NoneType' object has no attribute 'children'

In [11]:
with handlers.HandlerTaskNode.platform(client, compiler, jinja_env):
    args = root.run_args

In [12]:
args

[{'role': 'assistant',
  'content': '<reasoning>\nFor a bot that writes poems about cars, I expect users to send messages like "Write a poem about Tesla Model S" or "Create a haiku about vintage Mustang".\nThe LLM needs to extract:\n1. Car details (make, model, year if specified)\n2. Poetry style (if specified, otherwise default)\n3. Tone/mood (if specified)\n\nThe main function would be createPoem which takes these parameters.\nAdditional function getPoemHistory to retrieve previous poems about specific cars.\n\nThe Car model should contain basic car details, and Poem model should contain both the poem text and metadata about the style and tone.\n</reasoning>\n\n<typespec>\nmodel Car {\n    make: string;\n    model: string;\n    year: integer;\n}\n\nmodel Poem {\n    car: Car;\n    text: string;\n    style: string;\n    tone: string;\n    createdAt: utcDateTime;\n}\n\ninterface CarPoetBot {\n    @llm_func(2)\n    createPoem(car: Car, style: string, tone: string): Poem;\n\n    @llm_fun